In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:01<00:00, 6.87MB/s]


In [38]:
driver.get('https://maps.unimelb.edu.au/parkville/building')
#driver.get('https://maps.unimelb.edu.au/werribee/building')
#driver.get('https://maps.unimelb.edu.au/southbank/building')
content = driver.page_source
soup = BeautifulSoup(content, 'html.parser')

In [39]:
print(soup.prettify())

<html class="" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <link href="https://matrix-cms.unimelb.edu.au/__data/assets/image/0026/92564/favicon.png" rel="icon" sizes="16x16" type="image/png"/>
  <title>
   Southbank buildings
  </title>
  <!--
Website maintained by: aharris@unimelb.edu.au
Last updated by: Andrew Harris on 12 Apr 2019 3:04pm
Last published by: Never Published on 21 Jun 2016 5:02pm
System: MCE Gen2
    This website uses the centrally managed Design and Content Management System Gen 2.  
    https://matrix-cms.unimelb.edu.au/web
    Asset ID: #30997

-->
  <meta content="Southbank buildings " name="title"/>
  <meta content="" name="description"/>
  <meta content="" name="robots"/>
  <meta content="noodp" name="robots"/>
  <link href="https://maps.unimelb.edu.au/southbank/building" rel="canonical"/>
  <meta content="Victoria, Australia, Ocea

In [40]:
Building = []
Urls = []
Building_no = []

In [41]:
# Find all <li> elements with the class 'cell Building' or 'cell Building Food and drink'
for d in soup.find_all('li',{'class' : ['cell Building', 'cell Building Food and drink']}):
    a_tag = d.find('a')  # find the first 'a' tag in the HTML
    #url = a_tag['href'] # get the value of the 'href' attribute
    #print(a_tag.text)
    building_name = a_tag.text.strip()  # get the text content of the tag and remove any leading/trailing white space
    #print(building_name)
    #Urls.append(url)
    Building.append(building_name)

In [42]:
for d in soup.findAll('span', {'title': 'Building Number'}):
    buildNo = d.text.strip() 
    Building_no.append(buildNo)

In [43]:
df_parkVille = pd.DataFrame({'Building name':Building,'No': Building_no})
df_parkVille.head(5)

,Building name,No
0,Melbourne Theatre Company - HQ (298),298
1,Elisabeth Murdoch Building (860),860
2,Film and Television (861),861
3,Music (862),862
4,The Hub (863),863


In [44]:
# Remove the last 6 characters from all values in the "Building name" column
df_parkVille['Building name'] = df_parkVille['Building name'].apply(lambda x: x[0:len(x)-6])
df_parkVille.head(5)


,Building name,No
0,Melbourne Theatre Company - HQ,298
1,Elisabeth Murdoch Building,860
2,Film and Television,861
3,Music,862
4,The Hub,863


In [45]:
#Check blank building name
id_blank_bd_name = df_parkVille['Building name']==''

# Define a lambda function to update the URL string
def update_building_name(row):
    return ('Building ' +row['No'])

# Apply the lambda function to the "no" column and assign the results to a new column "url"
df_parkVille.loc[id_blank_bd_name,'Building name'] = df_parkVille.apply(update_building_name, axis=1)
df_parkVille.head(5)


,Building name,No
0,Melbourne Theatre Company - HQ,298
1,Elisabeth Murdoch Building,860
2,Film and Television,861
3,Music,862
4,The Hub,863


In [46]:
#Create campus id mapping
campus_id_mapping = pd.DataFrame(data={'Campus_code': ['WER', 'DOO','SPT','STH','BUR','CRE','PAR'], 'campus_id': [217, 220,221,216,218,219,200]})
campus_id_mapping


,Campus_code,campus_id
0,WER,217
1,DOO,220
2,SPT,221
3,STH,216
4,BUR,218
5,CRE,219
6,PAR,200


In [47]:
# Look up the campus_id for a given campus_code
campus_code = 'PAR'
#campus_code = 'WER'
#campus_code = 'STH'
campus_id = campus_id_mapping.loc[campus_id_mapping['Campus_code'] == campus_code, 'campus_id'].iloc[0]
campus_id

216

In [48]:
# Define a lambda function to update the URL string
def update_url(row):
    return 'https://use.mazemap.com/?campusid={}&sharepoitype=identifier&sharepoi={}'.format(campus_id, row['No'])

# Apply the lambda function to the "no" column and assign the results to a new column "url"
df_parkVille['url'] = df_parkVille.apply(update_url, axis=1)

# Print the updated dataframe
df_parkVille['url'].head(5)

0    https://use.mazemap.com/?campusid=216&sharepoi...
1    https://use.mazemap.com/?campusid=216&sharepoi...
2    https://use.mazemap.com/?campusid=216&sharepoi...
3    https://use.mazemap.com/?campusid=216&sharepoi...
4    https://use.mazemap.com/?campusid=216&sharepoi...
Name: url, dtype: object

Export to Excel

In [49]:
df_parkVille.to_excel("unimelb_parville_url.xlsx", index=False) 
#df_parkVille.to_excel("unimelb_werribee_url.xlsx", index=False) 
#df_parkVille.to_excel("unimelb_southBank_url.xlsx", index=False)